# 이미지 증강

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

from glob import glob
from tqdm import tqdm

from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2

from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [7]:
# 데이터 증강
from keras.preprocessing.image import array_to_img, img_to_array, load_img

# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 32

# Training 수
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 1024
img_height = 1024

# Online-augmentation 적용 Generator
# 1. 이미지를 전부다 불러서 램 (메모리)에 올릴 수 없기 때문
# 2. 이미지는 Augmentation을 해주는게 좋아서

DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=70,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

## train_status폴더 따로 만들고 train_status -> class별 폴더 -> class별 status 폴더 - 분류된 이미지
## 이런 식으로 생성해놓고 코드 돌렸습니다
## ex) train_status -> bottle -> (bottle-good, bottle-broken-small ...)

In [4]:
# class 이름만 추출
train_df = pd.read_csv("train_df.csv")
list_class = train_df["class"].unique().tolist()
list_class.sort()
list_class

CurrentDirectory = "./"
test_directory  = CurrentDirectory + 'test/'


list_train_directory = []
list_model_directory = []

# class 이름별 경로 리스트 생성(list_train_directory)
for i in list_class:
    list_train_directory.append(CurrentDirectory + 'train_status/' + i + '/')
    list_model_directory.append(CurrentDirectory + 'model_status/' + i + '/')

# class 이름별 경로 bottle부터 하나씩 뽑아서 순서대로 증강
for z in range(len(list_train_directory)):
    # 리스트에서 뽑은 경로
    path = list_train_directory[z]
    # 해당 class의 status별 리스트 a
    a = os.listdir(path)
    # 해당 class 이름만 추출
    e = list_train_directory[z][15:-1]
    b = []
    # 해당 class 이름과 '-good' 결합하여 증강시킬 폴더에서 제외
    eg = e + '-good'
    # b리스트 속 리스트로 각 상태별 파일 이름 추가
    for i in a:
        if i != eg:
            b.append(os.listdir(path + i))
    # 해당 class-good은 a리스트에서 삭제
    a.remove(eg)
    # 해당 class의 status 갯수만큼 b리스트 속 리스트 존재
    for k in range(len(b)):
        # status 속 파일리스트 d로 지정
        d = b[k]
        for i in a:
            c = os.listdir(path + i)
            directory = path + i + '/'
            # 이중 for문에서 폴더 접근 방식 에러 발생할 수 밖에 없어서 try문으로 에러 무시
            try:
                # 원본 파일리스트 d에서 하나씩 꺼내서 x에 array형태로 넣은 다음 reshape
                for j in d:
                    img = load_img(directory + j)
                    x = img_to_array(img)
                    x = x.reshape((1,) + x.shape)
                    p = 0
                    # array 형태로 변형된 원본 이미지 해당 경로에 앞에 tri 이름붙은 png 파일 형태로 
                    # ImageGenerator로 증강시키는 동작 i만큼 반복
                    for batch in DATAGEN_TRAIN.flow(x, batch_size=1, save_to_dir= directory, save_prefix='tri', save_format='png'):
                        p += 1
                        if p > 7:
                            break
            except:
                continue

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './train/bottle/'

In [8]:

img = load_img("./train/screw-scratch_head/10197.png")
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
p = 0
                    # array 형태로 변형된 원본 이미지 해당 경로에 앞에 tri 이름붙은 png 파일 형태로 
                    # ImageGenerator로 증강시키는 동작 i만큼 반복
for batch in DATAGEN_TRAIN.flow(x, batch_size=1, save_to_dir="./train/screw-scratch_head" , save_prefix='tri', save_format='png'):
    p += 1
    if p >= 3:
        break